This notebook illustrates the agent creation process for the **LLM 20 Questions**. Running this notebook produces a `submission.tar.gz` file. You may submit this file directly from the **Submit to competition** heading to the right. Alternatively, from the notebook viewer, click the *Output* tab then find and download `submission.tar.gz`. Click **Submit Agent** at the upper-left of the competition homepage to upload your file and make your submission. 

In [1]:
%%bash
cd /kaggle/working
pip install -q -U -t /kaggle/working/submission/lib immutabledict sentencepiece
git clone https://github.com/google/gemma_pytorch.git > /dev/null
mkdir /kaggle/working/submission/lib/gemma/
mv /kaggle/working/gemma_pytorch/gemma/* /kaggle/working/submission/lib/gemma/

Cloning into 'gemma_pytorch'...


### Definition

In [2]:
%%writefile submission/main.py

import sys
import os

print(os.getcwd())

KAGGLE_AGENT_PATH = "/kaggle_simulations/agent/"
if os.path.exists(KAGGLE_AGENT_PATH):
    sys.path.insert(0, os.path.join(KAGGLE_AGENT_PATH, 'lib/gemma'))
    sys.path.insert(0, os.path.join(KAGGLE_AGENT_PATH, 'lib'))
else:
    sys.path.insert(0, "/kaggle/working/submission/lib/gemma")
    sys.path.insert(0, "/kaggle/working/submission/lib/")

from typing import Iterable
from typing import Any, List, Optional, Sequence, Tuple, Union
from ast import parse
import contextlib, time, random, itertools, re
import torch
import gemma
from gemma.config import get_config_for_7b, get_config_for_2b
from gemma.model import GemmaForCausalLM, GemmaModel


VARIANT = '7b-it-quant'
MACHINE_TYPE = 'cuda'
ENV = 'kaggle'
agent = None
temperature = 1


# configering setting
sys_prompt_guesser = 'You are an AI assistant designed to play the 20 Questions game. \
In each round of the game, choose an attribute most likely to aid your deduction and ask a yes-no question based on it. \
Your response should be generated with step-by-step reasoning. Ensure that the question you generate has not been asked before and that its attribute is not shown in the known information list. Consider whether this attribute or a similar question has been asked before selecting key attributes. \
An unavailable question will cause seriouse problem so think more thoroughly before responsing. \
Once you believe that the key attribute is available, the most helpful attribute among effective ones provides the maximum information gain, meaning it significantly reduces the entropy of your guess, regardless of whether the answer is **yes** or **no**. Finally using this attribute, ask a question in English. \
Here are few examples of how you use come up with a formatted response with reasoning, remember they are just examples, please do not imitate its asking strategy:'

sys_prompt_answerer = f'You are an AI assistant designed to play the 20 Questions game. In this game, the Answerer thinks of a keyword and responds to yes-or-no questions by the Questioner. The keyword is a specific person, place, or thing. Here is an example of answering a yes-no question:'

few_shot_examples_answerer = [
    
'''
The keyword is Sydney in the category place. Give yes-or-no answer and surround your answer with double asterisks, like **yes** or **no**.

''',
     
'''
Question: Is it a place?
**yes**
Sydney is a city, which is a type of place. This is why the answer to the question "Is it a place?" is yes.
''',
     
'''
Question: Is it located in the Southern Hemisphere?
**yes**
Sydney is a city in Australia, which is in the Southern Hemisphere. Therefore, the answer to your question is yes.
''',
     
'''
Question: Is it a capital city?
**no**
Although Sydney is a major city in Australia, the capital city of Australia is Canberra. Therefore, the answer to your question is no.
''',
     
     
'''
Question: Is it a well-known tourism city?
**yes**
Sydney is famous for its landmarks such as the Sydney Opera House, the Harbour Bridge, and its beautiful beaches, making it a major tourist destination. Therefore, the answer to your question is yes.
'''
                              
]

few_shot_examples_ask = [
    
[# Round 1 | Keyword: Yangtze River
'''
Known information: ["is it a country? --> no", "is it a city? --> no"]
**Key attribute**: "man-made structure" 
**Redundent attribute**: no, since following information havn't been confirmed according to the known information list.
**Question**: "Is it a man-made structure?" 
**Reasoning**:
since following information havn't been confirmed according to the known information list, so it's legal and avaliable to consider that: 
Man-made vs. Natural: This question helps to differentiate between natural landmarks (e.g., Grand Canyon) and man-made structures (e.g., Eiffel Tower). 
Broad Categories: Man-made structures include a wide range of possibilities (e.g., buildings, monuments), whereas cities are all man-made but represent a different category. 
''',

'''
Known information: ["is it a country? --> no", "is it a city? --> no", "is it a landmark? --> yes", "is it a man-made structure? --> no", "is it in North America? --> no"]
**Key attribute**: "in Asia" 
**Redundent attribute**: no, since following information havn't been confirmed according to the known information list.
**Question**: "Is it located in Asia?" 
**Reasoning**:
since following information havn't been confirmed according to the known information list, so it's legal and avaliable to consider that: 
Geographic Focus: Identifying the continent will significantly narrow down the possible natural landmarks. 
Large Landmarks: Asia has several major natural landmarks (e.g., Mount Everest, Great Wall).
''',

'''
Known information: ["is it a country? --> no", "is it a city? --> no", "is it a landmark? --> yes", "is it a man-made structure? --> no", "is it in North America? --> no", "is it in Aisa? --> yes", "is it in China? --> yes", "is it a river? --> yes"]
**Key attribute**: "longest river in China"
**Redundent attribute**: no, since following information havn't been confirmed according to the known information list.
**Question**: "Is it the longest river in China?"
**Reasoning**:
since following information havn't been confirmed according to the known information list, so it's legal and avaliable to consider that: 
Length-Specific Focus: Knowing whether it is the longest river will help confirm or eliminate the Yangtze River.
Major Rivers: The Yangtze River is the longest river in China, followed by the Yellow River.
Effective Split: This question will help confirm or deny one of the most prominent landmarks in China, providing a clear yes/no split.
'''],


[# Round 2 | Keyword: Congo
'''
Known information: ["is it a country? --> yes"]
**Key attribute**: "in Europe"
**Redundent attribute**: no, since following information havn't been confirmed according to the known information list.
**Question**: "Is this country in Europe?"
**Reasoning**:
since following information havn't been confirmed according to the known information list, so it's legal and avaliable to consider that: 
Continental Focus: Knowing the continent will help significantly narrow down the list of possible countries.
Effective Reduction: Europe has many countries, so confirming or eliminating Europe will reduce the search space.
''',

'''
Known information: ["is it a country? --> yes", "is it in Europe? --> no", "is it in Asia? --> no", "is it in Africa? --> yes", "is it in Northern Africa? --> no", "is it in Eastern Africa? --> no", "is it in Western Africa? --> no", "is it in Southern Africa? --> yes"]
**Key attribute**: "landlocked country"
**Redundent attribute**: no, since following information havn't been confirmed according to the known information list.
**Question**: "Is this country landlocked?"
**Reasoning**:
To further narrow down the possibilities within Southern Africa, I will focus on another well-known country in that region.
Geographical Focus: Knowing whether the country is landlocked will significantly narrow down the possibilities within Southern Africa.
Effective Reduction: There are several landlocked countries in Southern Africa, so confirming or eliminating this will help focus the search.
''',

'''
Known information: ["is it a country? --> yes", "is it in Europe? --> no", "is it in Asia? --> no", "is it in Africa? --> yes", "is it in Northern Africa? --> no", "is it in Eastern Africa? --> no", "is it in Western Africa? --> no", "is it in Southern Africa? --> yes", "is it landlocked? --> no", "is Portuguese its an official language? --> no"]
**Key attribute**: "borders the Indian Ocean"
**Redundent attribute**: no, since following information havn't been confirmed according to the known information list.
**Question**: "Does this country border the Indian Ocean?"
**Reasoning**:
since following information havn't been confirmed according to the known information list, so it's legal and avaliable to consider that: 
Geographic Focus: Knowing whether the country borders the Indian Ocean can help narrow down the options.
Balanced Distribution: This question provides a clear yes/no split, effectively narrowing down the search.
 '''],
 
[# round 3 | Keyword: Ryan 
'''
Known information: ["is it a place? --> no", "is it a person? --> yes"]
**Key attribute**: "a historical figure"
**Redundent attribute**: no, since following information havn't been confirmed according to the known information list.
**Question**: "Is this person a historical figure?"
**Reasoning**:
Time Frame: Identifying whether the person is historical can significantly narrow down the possibilities.
Balanced Distribution: This question provides a clear yes/no split, guiding the search effectively.
''',

'''
Known information: ["is it a place? --> no", "is it a person? --> yes", "is it a historical figure? --> no", "is it involved in the entertainment industry? --> yes", "is it primarily known for their work in music? --> no", "is it primarily known for their work as an actor? --> yes", "has it won an Academy Award (Oscar)? --> no"]
**Key attribute**: "primarily known for television work"
**Redundent attribute**: no, since following information havn't been confirmed according to the known information list.
**Question**: "Is this person primarily known for their work in television?"
**Reasoning**:
Medium Focus: Identifying whether the person is known for television can help narrow down the possibilities.
Balanced Distribution: This question provides a clear yes/no split, effectively guiding the search.
''',

'''
Known information: ["is it a place? --> no", "is it a person? --> yes", "is it a historical figure? --> no", "is it involved in the entertainment industry? --> yes", "is it primarily known for their work in music? --> no", "is it primarily known for their work as an actor? --> yes", "has it won an Academy Award (Oscar)? --> no", "is it primarily known for their work in television? --> no", "is it primarily known for their work in action movies? --> yes", "is it associated with a major action movie franchise? --> yes"]
**Key attribute**: "known for science fiction action movies"
**Redundent attribute**: no, since following information havn't been confirmed according to the known information list.
**Question**: "Is this person known for their work in science fiction action movies?"
**Reasoning**:
Genre Specificity: Identifying whether the person is known for science fiction action movies can help narrow down the possibilities.
Effective Reduction: This helps distinguish between different types of action movie genres, such as sci-fi, fantasy, or military.
'''],

[# round 4 | Keyword: Noosa
'''
Known information: ["is it a person? --> no", "is it a place? --> yes", "is it a country? --> no", "is it a city? --> yes"]
**Key attribute**: "a capital city"
**Redundent attribute**: no, since following information havn't been confirmed according to the known information list.
**Question**: "Is it a capital city?"
**Reasoning**:
Geographic Focus: This question helps determine if the city is a capital, significantly narrowing the possibilities.
Effective Reduction: If the answer is yes, it focuses on capital cities, eliminating all non-capital cities. If no, it eliminates all capital cities from consideration.
Context Relevance: Knowing whether the city is a capital helps tailor subsequent questions to specific types of cities.
''',

'''
Known information: ["is it a person? --> no", "is it a place? --> yes", "is it a country? --> no", "is it a city? --> yes", "is it a capital city? --> no", "is it in the Southern Hemisphere? --> yes."]
**Key attribute**: "in the Southern Hemisphere"
**Redundent attribute**: no, since following information havn't been confirmed according to the known information list.
**Question**: "Is this city in the Southern Hemisphere?"
**Reasoning**:
Hemispheric Focus: This question splits the world into two large, nearly equal parts, effectively narrowing down possibilities.
Balanced Distribution: The Earth is divided evenly by the equator, providing a clear yes/no split that maximizes information gain.
''',


'''
Known information: ["is it a person? --> no", "is it a place? --> yes", "is it a country? --> no", "is it a city? --> yes", "is it a capital city? --> no", "is it in the Southern Hemisphere? --> yes", "is it in Africa? --> no", "is it in South America? --> no", "is it in Australia or Oceania? --> yes", "is it in Australia? --> yes", "is it a coastal city? --> yes"]
**Key attribute**: "a well-known tourist destination"
**Redundent attribute**: no, since following information havn't been confirmed according to the known information list.
**Question**: "Is this city a well-known tourist destination?"
**Reasoning**:
Tourism Focus: This question helps determine if the city is popular with tourists, which can narrow down the list of coastal cities.
Effective Reduction: If the answer is yes, it focuses on well-known tourist cities, eliminating less-known cities. If no, it narrows down to less-touristic coastal cities.
Context Relevance: Many of Australia's coastal cities are also popular tourist destinations, making this an important distinction.
''',

'''
Known information: ["is it a person? --> no", "is it a place? --> yes", "is it a country? --> no", "is it a city? --> yes", "is it a capital city? --> no", "is it in the Southern Hemisphere? --> yes", "is it in Africa? --> no", "is it in South America? --> no", "is it in Australia or Oceania? --> yes", "is it in Australia? --> yes", "is it a coastal city? --> yes, "is a well-known tourist destination? --> yes", "is it Gold Coast? --> no", "is it in New South Wales? --> no", "is it in Queensland? --> yes", "is it associated with the Great Barrier Reef? --> no"]
**Key attribute**: "popular for its beaches"
**Redundent attribute**: no, since following information havn't been confirmed according to the known information list.
**Question**: "Is this city popular for its beaches?"
**Reasoning**:
Tourism Focus: Identifying if the city is known for its beaches can help narrow down the possibilities.
Context Relevance: Many coastal cities in Queensland are known for their beaches, making this a significant distinction.
Balanced Distribution: Coastal versus inland tourist destinations provide a clear yes/no split.
''',

'''
Known information: ["is it a person? --> no", "is it a place? --> yes", "is it a country? --> no", "is it a city? --> yes", "is it a capital city? --> no", "is it in the Southern Hemisphere? --> yes", "is it in Africa? --> no", "is it in South America? --> no", "is it in Australia or Oceania? --> yes", "is it in Australia? --> yes", "is it a coastal city? --> yes, "is a well-known tourist destination? --> yes", "is it Gold Coast? --> no", "is it in New South Wales? --> no", "is it in Queensland? --> yes", "is it associated with the Great Barrier Reef? --> no", "is it popular for its beaches? --> yes"]
**Key attribute**: "located on the Sunshine Coast"
**Redundent attribute**: no, since following information havn't been confirmed according to the known information list.
**Question**: "Is this city located on the Sunshine Coast?"
**Reasoning**:
Regional Focus: Identifying if the city is on the Sunshine Coast can significantly narrow down the possibilities.
Context Relevance: The Sunshine Coast is known for its beach destinations, making this a significant distinction.
'''],

[# round 5 | Keyword: Gym Mat
'''
Known information: ["is it a person? --> no", "is it a place? --> no", "is it a thing? --> yes"]
**Key attribute**: "a technological device"
**Redundent attribute**: no, since following information havn't been confirmed according to the known information list.
**Question**: "Is it a technological device?"
**Reasoning**:
Category Specificity: Identifying whether the thing is a technological device helps narrow down the possibilities.
Effective Reduction: This helps focus on a specific type of thing.
''',

'''
Known information: ["is it a person? --> no", "is it a place? --> no", "is it a thing? --> yes", "is it a technological device? --> no"]
**Key attribute**: "found in nature"
**Redundent attribute**: no, since following information havn't been confirmed according to the known information list.
**Question**: "Is it something found in nature?"
**Reasoning**:
Category Specificity: Identifying whether the thing is natural helps narrow down the possibilities.
Balanced Distribution: This question provides a clear yes/no split, guiding the search effectively.
''',

'''
Known information: ["is it a person? --> no", "is it a place? --> no", "is it a thing? --> yes", "is it a technological device? --> no", "can it be found in nature? --> no"]
**Key attribute**: "found in households"
**Redundent attribute**: no, since following information havn't been confirmed according to the known information list.
**Question**: "Is it something found in households?"
**Reasoning**:
Category Specificity: Identifying whether the thing is commonly found in households helps narrow down the possibilities.
Effective Reduction: This helps focus on a specific type of man-made object.
Balanced Distribution: This question provides a clear yes/no split, guiding the search effectively.
''']
]

few_shot_examples_guess = [
'Known information: ["is it a country? --> no", "is it a city? --> no", "is it a landmark? --> yes"]\n\
Given these clues, it is likely a natural landmark rather than a city or a man-made structure. An example of a well-known natural landmark is the Grand Canyon.\
Guess: "Grand Canyon"'

'Known information: ["is it a country? --> no", "is it a city? --> no", "is it a landmark? --> yes", "is it a man-made structure? --> no"]\n\
Given these clues, it is likely a natural landmark outside North America. An example of a famous natural landmark outside North America is Mount Everest.\
Guess: "Mount Everest"'
 
'Known information: ["is it a country? --> no", "is it a city? --> no", "is it a landmark? --> yes", "is it a man-made structure? --> no", "is it in North America? --> no"]\n\
Given these clues, it is likely a natural landmark in Asia. A famous natural landmark in Asia is the Himalayas.\
Guess: "the Himalayas"'
 
'Known information: ["is it a country? --> no", "is it a city? --> no", "is it a landmark? --> yes", "is it a man-made structure? --> no", "is it in North America? --> no", "is it in Aisa? --> yes"]\n\
Given these clues, it is likely a famous natural landmark in China. An example of such a landmark is the Mekong River.\
Guess: "the Mekong River"'
 
'Known information: ["is it a country? --> no", "is it a city? --> no", "is it a landmark? --> yes", "is it a man-made structure? --> no", "is it in North America? --> no", "is it in Aisa? --> yes", "is it in China? --> yes"]\n\
Given these clues, it is likely a natural landmark in China that is not a mountain. An example of such a landmark could be the Tianchi Lake.\
Guess: "the Tianchi Lake"'
 
'Known information: ["is it a country? --> no", "is it a city? --> no", "is it a landmark? --> yes", "is it a man-made structure? --> no", "is it in North America? --> no", "is it in Aisa? --> yes", "is it in China? --> yes", "is it a river? --> yes"]\n\
Given these clues, a very famous river in China is the Yellow River.\
Guess: "the Yellow River"',
                          
'Known information: ["is it a country? --> no", "is it a city? --> no", "is it a landmark? --> yes", "is it a man-made structure? --> no", "is it in North America? --> no", "is it in Aisa? --> yes", "is it in China? --> yes", "is it a river? --> yes", "is it the longest river in China? --> yes"]\n\
Given these clues, it should be the longest river in China which is the Yangtze River.\
Guess: "the Yangtze River"']

@contextlib.contextmanager
def _set_default_tensor_type(dtype: torch.dtype):
  """Sets the default torch dtype to the given dtype."""
  torch.set_default_dtype(dtype)
  yield
  torch.set_default_dtype(torch.float)

def interleave_unequal(x, y):
    '''
        Interleave two lists of unequal length.
    '''
    return [
        item for pair in itertools.zip_longest(x, y) for item in pair if item is not None
    ]


# Formatter Definition
class PromptFormatter:
    
    '''
        formatter class to format the prompt text for the model. 
        A general idea is 
    '''
    
    _start_token = '<start_of_turn>'
    _end_token = '<end_of_turn>'
        
    def __init__(self, system_prompt: str = None, few_shot_examples: Iterable = None):
        self._system_prompt = system_prompt
        self._few_shot_examples = few_shot_examples
        self._template_user = f"{self._start_token}user\n{{}}{self._end_token}\n"
        self._template_model = f"{self._start_token}model\n{{}}{self._end_token}\n"
        self._all_prompt = ''
        # if sample_num is not None:
        #   self.sample_num = sample_num
        # elif self._few_shot_examples is not None:
        #   self.sample_num = len(few_shot_examples)
        # else:
        #   self.sample_num = None

        self.reset()

    def __repr__(self):
        return self._all_prompt
        
    def reset(self):
        self._all_prompt = ''
        # if system prompt is provided, add it to the prompt
        if self._system_prompt is not None:
            self._all_prompt += self._template_user.format(self._system_prompt)
        # same for few shot examples
        if self._few_shot_examples is not None:
            # self.add_rounds(self._few_shot_examples, start_agent='user')
            self.add_new_round('user', 'Here are some examples of analyzing with causal reasoning:', True)
            # for example in random.sample(self._few_shot_examples, self.sample_num):
            for example in self._few_shot_examples:
                self.add_new_round('model', example.strip(), True)
            
        
    def add_user_round(self, user_prompt: str):
        # add user round to the prompt
        self._all_prompt += self._template_user.format(user_prompt)

    def add_agent_round(self, model_response: str):
        self._all_prompt += self._template_model.format(model_response)
    
    def add_rounds(self, rounds: Iterable, start_agent: str):
        '''
            Apply a sequence of rounds to the formatter, starting with the specified agent.
        '''
        formatters = [self.add_agent_round, self.add_user_round] if start_agent == 'model' else [self.add_user_round, self.add_agent_round] # here, self.model and self.user are functions definded above
        formatters = itertools.cycle(formatters)
        for fmt, round in zip(formatters, rounds):
            fmt(round)
        return self
    
    # def add_end_token(self):
    #     self._all_prompt += f"{self._end_token}\n"
    
    def add_new_round(self, player: str, prompt:str = None, end_token: bool = False):
        self._all_prompt += f"{self._start_token}{player}\n"
        if prompt is not None:
            self._all_prompt += f'{prompt}'
        if end_token:
            # self.add_end_token()
            self._all_prompt += f"{self._end_token}\n"
    
    def formate_MCQA(self):
        raise NotImplementedError
    
# print(str(PromptFormatter(sys_prompt, few_shot_examples_ask)))
        
# Agent Definition
class GemmaAgent:
    def __init__(self, model_variant, device='cuda:0', env="kaggle", output_len=200, system_prompt=None, few_shot_examples=None):
        # model initialization
        self.device = device
        self.model_variant = model_variant
        
        WEIGHTS_PATH = self._set_up_env(env)
                
        # Ensure that the tokenizer is present
        tokenizer_path = os.path.join(WEIGHTS_PATH, 'tokenizer.model')
        assert os.path.isfile(tokenizer_path), 'Tokenizer not found!'
        
        # Ensure that the checkpoint is present
        ckpt_path = os.path.join(WEIGHTS_PATH , f'gemma-{model_variant}.ckpt')
        assert os.path.isfile(ckpt_path), 'PyTorch checkpoint not found!'

        # loading model configuration
        model_config = get_config_for_2b() if "2b" in model_variant else get_config_for_7b()
        model_config.quant = "quant" in model_variant
        model_config.tokenizer = os.path.join(WEIGHTS_PATH, "tokenizer.model")
        
        with _set_default_tensor_type(model_config.get_dtype()):
            self.model = GemmaForCausalLM(model_config)
            self.model.load_weights(ckpt_path)
            self.model = self.model.to(self.device).eval()
            
        self.round_num = 0
        self.known_info = []
        self.last_key_attribute = None
        self.last_guess = None
        self.output_len = output_len
        
        # formatter arguments   
        self.system_prompt = system_prompt
        self.few_shot_examples = few_shot_examples

    def _set_up_env(self, env):
        
        if env == 'kaggle':
            print("Loading model in Kaggle, model weights will be searched within local directories.")
            
            # kaggle configuration
            KAGGLE_AGENT_PATH = "/kaggle_simulations/agent/"
            if os.path.exists(KAGGLE_AGENT_PATH):
                WEIGHTS_PATH = os.path.join(KAGGLE_AGENT_PATH, f"gemma/pytorch/{self.model_variant}/2")
            else:
                WEIGHTS_PATH = f"/kaggle/input/gemma/pytorch/{self.model_variant}/2"
                            
        elif env == 'colab':
            print("Loading model in Colab, starting from downloading the model weights.")
            
            WEIGHTS_PATH = kagglehub.model_download(f'google/gemma/pyTorch/{self.model_variant}')
        else:
            raise ValueError("Argument 'env' should be in ['kaggle', 'colab']")

        return WEIGHTS_PATH

    def _parse_response(self, response : str):
      raise NotImplementedError
  
    def _format_prompt(self, obs):
      raise NotImplementedError


class GemmaAgent_Answer(GemmaAgent):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        # set up few_shot_examples
        self.formatter = PromptFormatter(system_prompt=self.system_prompt,
                                        few_shot_examples=self.few_shot_examples)

    def _parse_response(self, response : str):
        pattern = re.compile(r'\*\*([^*]+)\*\*', re.DOTALL)
        matches = pattern.findall(response.lower())
        
        flag_no = 'no' in matches
        flag_yes = 'yes' in matches
        
        # if both yes and no are in the response, randomly choose one
        if flag_no and not flag_yes:
            ret = 'no'
        elif flag_yes and not flag_no:
            ret = 'yes'
        else:
            ret = random.choice(['yes', 'no'])
        
        return ret

    def _format_prompt(self, obs):
        self.formatter.reset()
        self.formatter.add_new_round('user', f'Remember how to use CoT as above and the format of answering. Now let us play 20 Questions. You are playing the role of the Answerer. The keyword is {obs["keyword"]} in the category {obs["category"]}.', True)
        # add played rounds
        rounds = interleave_unequal(obs["questions"], obs["answers"])
        self.formatter.add_rounds(rounds, start_agent='user')
        self.formatter.add_new_round('user', 
                                     f'The keyword is {obs["keyword"]} in the category {obs["category"]}. Give yes-or-no answer and surround your answer with double asterisks, like **yes** or **no**.', 
                                     True)
        # start of model response
        self.formatter.add_new_round('model', f'Question: {obs["questions"][-1]}', False)
        
    def __call__(self, obs, cfg=None):
        print(obs['questions'])
        self._format_prompt(obs)
        prompt = str(self.formatter)
        # print(prompt)
        response = self.model.generate(prompt, device=self.device, output_len=self.output_len)
        print(f'===> Turn Type: {obs["turnType"]}\nResponse: {response}')
        return self._parse_response(response)


class GemmaAgent_Guesser(GemmaAgent):
    def __init__(self, *args, **kwargs):
        # start = time.time()
        # set up general configurations
        super().__init__(*args, **kwargs)
        # set up few_shot_examples
        self.few_shot_exmaples_ask = self.few_shot_examples[0]
        self.few_shot_exmaples_guess = self.few_shot_examples[1]
        # agent args
        self.formatter = PromptFormatter(system_prompt=self.system_prompt, 
                                        few_shot_examples=self.few_shot_exmaples_guess)
        # print(f'Initialization Time: {time.time()-start}s')
        self.categories = ['person', 'thing', 'place']

        
    
    def _parse_response(self, response : str):
        '''
            parse the response into a dictionary.
            may contain three keys: key_attribute, question, guess and their value respectively.
            e.g.: {"key_attribute": 'a country', "question": 'Is it a country?'} for a parsed asker response.
        '''
        pattern = re.compile(r'\*\*([^*]+)\*\*:?\s*(.*?)(?=\*\*|$)', re.DOTALL)
        matches = pattern.findall(response.lower())
        parse_dict = {'key attribute':None, 'question':None, 'guess':None}
        for k, v in matches:
            v = v.strip('\n\'\"')
            if ('attr' in k) and (parse_dict['key attribute'] == None):
                parse_dict['key attribute'] = v
            elif ('ques' in k) and (parse_dict['question'] == None):
                parse_dict['question'] = v
            elif ('guess' in k) and (parse_dict['guess'] == None):
                parse_dict['guess'] = v

        return parse_dict
    
    def _format_prompt(self, obs):
        if obs["turnType"] == 'ask':
          self.formatter._few_shot_examples = random.choice(self.few_shot_exmaples_ask)
        if obs["turnType"] == 'guess':
          self.formatter._few_shot_examples = self.few_shot_exmaples_guess
        # reset formatter
        self.formatter.reset()
        self.formatter.add_new_round('user', 'Remember how to use CoT as above and the format of answering, now let us play a new game from the begining.', True)
        # add played rounds
        rounds = interleave_unequal(obs["questions"], obs["answers"])
        self.formatter.add_rounds(rounds, start_agent='model')
        if obs["turnType"] == 'ask':
            self.formatter.add_new_round('user', 
                                         'Now it is your turn to ask a question. Please construct a new question with chain of thought. Please consider **redundent attribute** step by step and ensure that your chosen attribute is not redundent, which means that your attribute is not redundent and should not be asked before. Baiscally it should follows the format within 100 words as following: **Key attribute**:"a summarized attribute about your question"\n**Redundent**:"whether and why it is a key attribute that you have not asked before"\n**Question**:"a yes-no question deducted from your reasoning"\n**Reasoning**:"your reasoning about why this attribute is avaliable and the most helpful"', 
                                         True)
        elif obs["turnType"] == 'guess':
            self.formatter.add_new_round('user', 
                                         'Now guess the keyword based on your analysis with reasoning and known information. And surround a pointer of guess with double asterisks (**?**) and your guessed keyword with doublr quotation markers ("?") in the end, such as **Guess** "your guess"', 
                                         True)
        # start of model response
        self.formatter.add_new_round('model', f'Given information: {str(self.known_info)}', False)                

    def __call__(self, obs, cfg=None):
        '''
            Main function to interact with the model. 
                1. update known information based on the observation and round number
                2. format the prompt with the observation
                3. generate response from the model
                4. parse the response and update information if needed
        '''

        if obs["turnType"] == 'ask': 
            print(f"=========== round: {self.round_num} | last key attr: {self.last_key_attribute}===========")
            self.round_num += 1
            
            # the category still remains unknown
            if isinstance(self.categories, list) and len(self.categories) > 1:
                # for the first round, randomly choose an attribute as initialization
                self.last_key_attribute = random.choice(self.categories)
                # kick out the selected key attribute from the categories
                self.categories = list(set(self.categories) - set([self.last_key_attribute]))
                response = f'Is it a {self.last_key_attribute}?'
                return response
            
            # Formatting prompt with observations
            self._format_prompt(obs)
            prompt = str(self.formatter)
            # debug
            print(f"===> Turn Type: {obs['turnType']}")
            print("==> Known Info:", self.known_info)
            # Getting response from LLM
            response = self.model.generate(prompt, device=self.device, output_len=self.output_len)   
            print(f'=> Response: {response}')
            
            # parse response and update information if needed
            parse_dict = self._parse_response(response)
            # print(f'\nParse_dict: {parse_dict}')
            # if in an ask turn, try to grab the key attribute from the response and update last key attribute for the next information updating
            # successfully grab the key attribute from the response
            if parse_dict['key attribute']:
                last_key_attribute = parse_dict['key attribute'].lower()
                last_key_attribute = last_key_attribute.replace('is not ', '')
                last_key_attribute = last_key_attribute.replace('is ', '')
                self.last_key_attribute = last_key_attribute
            # no key attribute parsed from the response, directly use question as key attribute
            elif parse_dict['question']:
                self.last_key_attribute  = parse_dict['question']
                ret = parse_dict['question']
                return ret
            # worst case, there is neither key attribute nor question parsed from the response, randomly choose an attribute
            else:
                random_alphabet = random.choice('abcdefghijklmnopqrstuvwxyz')
                ret = f'is it started with alphabet {random_alphabet}?'
                self.last_key_attribute = f'started with alphabet {random_alphabet}'
                return ret

            # parsed response has question, use question as response
            if parse_dict['question']:
                ret = parse_dict['question']
            # parsed response has no question but have key attribute, use key attribute as question
            else:

                ret = f'is it {self.last_key_attribute}?'   
            return ret
        # if in a guess turn, try to grab the guess from the response and update last guess for the next information updating
        elif obs["turnType"] == 'guess':
            last_question = obs['questions'][-1]
            # update information
            if isinstance(self.categories, list) and len(self.categories) > 0:
                # if it is the second round, grab the answer and update known information
                last_answer = obs["answers"][-1]
                # can determine the category
                if last_answer == 'yes':
                    self.known_info.append(f'is it a {self.last_key_attribute}? --> yes')
                    self.categories = self.last_key_attribute
                # the category is the one left in the category list
                elif len(self.categories) == 1:
                    self.known_info.append(f'is it a {self.last_key_attribute}? --> no')
                    self.categories = self.categories[0]
                    self.known_info.append(f'is it a {self.categories}? --> yes')
                # there are still more than one categories left
                else:
                    self.known_info.append(f'is it a {self.last_key_attribute}? --> no')                
            else:
                # other rounds, update known information
                last_answer = obs['answers'][-1]
                self.known_info.append(f'{last_question} --> {last_answer}')
                # if last_answer == 'yes':
                    # self.known_info.append(f'{last_question} --> yes')
                # else:
                    # self.known_info.append(f'is not {self.last_key_attribute}')
            
            # Formatting prompt with observations
            self._format_prompt(obs)
            prompt = str(self.formatter)
            print(f"===> Turn Type: {obs['turnType']}")
            print("==> Known Info:", self.known_info)
            
            # print("++++++++++++++++++++++++++++++\n", prompt, "\n++++++++++++++++++++++++++++++")
            
            # Getting response from LLM
            response = self.model.generate(prompt, device=self.device, output_len=self.output_len)   
            print(f'=> Response: {response}')
            
            # parse response and update information if needed
            parse_dict = self._parse_response(response)
            # print(f'\nParse_dict: {parse_dict}')
            if parse_dict['guess']:
                self.last_guess = parse_dict['guess']
                ret = parse_dict['guess']
            # if there is no guess parsed from the response, return empty string
            else:
                ret = ''
                
            return ret
        else:
            raise ValueError('Invalid turnType.')
    
# Simulation API
def get_agent(name):
    global agent
    
    if agent is None and name == 'questioner':
        agent = GemmaAgent_Guesser(model_variant=VARIANT, 
                                   device=MACHINE_TYPE, 
                                   env=ENV, 
                                   output_len=100, 
                                   system_prompt=sys_prompt_guesser, 
                                   few_shot_examples=[few_shot_examples_ask, few_shot_examples_guess])
    elif agent is None and name == 'answerer':
        agent = GemmaAgent_Answer(model_variant=VARIANT, 
                                  device=MACHINE_TYPE, 
                                  env=ENV, 
                                  output_len=50, 
                                  system_prompt=sys_prompt_answerer,
                                  few_shot_examples=few_shot_examples_answerer)
    assert agent is not None, 'Agent not found!'
    return agent

def get_fn(obs, config):
#     start = time.time()

    # print(f"\nStrp: {obs['step']}")
    # print(agent)

    if obs['turnType'] == "ask":
        response = get_agent('questioner')(obs)
    elif obs['turnType'] == "guess":
        response = get_agent('questioner')(obs)
    elif obs['turnType'] == "answer":
        response = get_agent('answerer')(obs)

    if response is None or len(response) <= 1:
        return "yes"
    else:
        # print(f'TOTAL TIME: {time.time()-start}s')
        # print(f'Turn Type: {obs["turnType"]}\nResponse: {response}')
        return response

Writing submission/main.py


In [3]:
!apt install pigz pv > /dev/null

In [4]:
!tar --use-compress-program='pigz --fast --recursive | pv' -cf submission.tar.gz -C /kaggle/working/submission . -C /kaggle/input/ gemma/pytorch/7b-it-quant/2